In [1]:
!wandb login ab1c1bd511648d87ba3c2fe1c223303b00cb4d24
import warnings
warnings.filterwarnings('ignore')

import os
import sys

import pandas as pd
import numpy as np
import torch
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor, ModelCheckpoint
from pytorch_forecasting import TimeSeriesDataSet, TemporalFusionTransformer
from pytorch_forecasting.data import GroupNormalizer
from pytorch_forecasting.metrics import QuantileLoss
from pytorch_lightning.loggers import WandbLogger

wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\miaoy/.netrc


In [2]:
args = {
    # ------------------------------
    # Basic config
    'random_seed': 11785,
    'n_samples': 1000,
    'batch_size': 64,
    'n_workers' : 4,
    # ------------------------------
    # Hyperparameters
    'lr_s': 7e-3,
    'hidden_size': 512,
    'attention_head_size': 2,        # use multihead for large hidden size
    'dropout': 0.1,
    'hidden_continuous_size': 4,     # set to <= hidden_size
    'output_size': 7,                # 7 quantiles for QuantileLoss by default
    'reduce_on_plateau_patience': 4, # reduce learning rate if no improvement in validation loss after x epochs
    'gradient_clip_val': 0.1,
    # ------------------------------
    # Logging
    'log_interval': 5,               # log every n batches, set to None when try to find best lr
    'wandb_project': 'miao-tft',
    'wandb_name': 'base',
}

In [3]:
train_dataset = TimeSeriesDataSet.load(os.path.join('C:/Users/miaoy/Desktop/11785/HwData/project/pf_train_1000_samples_pca.pf'))
val_dataset = TimeSeriesDataSet.load(os.path.join('C:/Users/miaoy/Desktop/11785/HwData/project/pf_val_1000_samples_pca.pf'))
test_dataset = TimeSeriesDataSet.load(os.path.join('C:/Users/miaoy/Desktop/11785/HwData/project/pf_test_1000_samples_pca.pf'))

batch_size = 32
train_dataloader = train_dataset.to_dataloader(train=True, batch_size=batch_size, num_workers=4)
val_dataloader = val_dataset.to_dataloader(train=False, batch_size=batch_size * 5, num_workers=4)

In [4]:
early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=1e-4, patience=10, verbose=False, mode="min")
lr_logger = LearningRateMonitor()  # log the learning rate

checkpoint_callback = ModelCheckpoint(
    monitor='val_SMAPE', 
    dirpath='C:/Users/miaoy/Desktop/11785/HwModel/project', 
    save_top_k=2, 
    filename='1000-default-{epoch:02d}-{val_SMAPE:.2f}'
)

logger = WandbLogger(
    project="miao_tft",
    name='pca',
    log_model=True
)

trainer = pl.Trainer(
    max_epochs=200,
    gpus=1,
    weights_summary="top",
    gradient_clip_val=0.1,
    limit_train_batches=30,  # coment in for training, running valiation every 30 batches
    # fast_dev_run=True,  # comment in to check that networkor dataset has no serious bugs
    callbacks=[lr_logger, early_stop_callback, checkpoint_callback],
    logger=logger,
)


tft_model = TemporalFusionTransformer.from_dataset(
    train_dataset,
    learning_rate=args['lr_s'],
    hidden_size=args['hidden_size'],  # most important hyperparameter apart from learning rate
    attention_head_size=args['attention_head_size'], # number of attention heads. Set to up to 4 for large datasets
    dropout=args['dropout'],  # between 0.1 and 0.3 are good values
    hidden_continuous_size=args['hidden_continuous_size'],
    output_size=args['output_size'],
    loss=QuantileLoss(),
    reduce_on_plateau_patience=args['reduce_on_plateau_patience'], # reduce learning rate if no improvement in validation loss after x epochs
)

# tft_model.load_state_dict(torch.load('C:/Users/miaoy/Desktop/11785/HwModel/project/tft_baseline_426.pth'))
# fit network
trainer.fit(
    tft_model,
    train_dataloaders=train_dataloader,
    val_dataloaders=val_dataloader,
)
torch.save(tft_model.state_dict(),'C:/Users/miaoy/Desktop/11785/HwModel/project/tft_baseline_429.pth')

wandb: Currently logged in as: auf2s (use `wandb login --relogin` to force relogin)


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name                               | Type                            | Params
----------------------------------------------------------------------------------------
0  | loss                               | QuantileLoss                    | 0     
1  | logging_metrics                    | ModuleList                      | 0     
2  | input_embeddings                   | MultiEmbedding                  | 77.0 K
3  | prescalers                         | ModuleDict                      | 928   
4  | static_variable_selection          | VariableSelectionNetwork        | 25.2 K
5  | encoder_variable_selection         | VariableSelectionNetwork        | 1.0 M 
6  | decoder_variable_selection         | VariableSelectionNetwork        | 1.0 M 
7  | static_context_variable_selection  | GatedResidualNetw

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]